# PCAI PDF RAG System

## Complete Implementation of 5 Required Components

This notebook demonstrates a working Retrieval-Augmented Generation system using PDF File

**Components Implemented:**
1. Knowledge Base (PDF)
2. Semantic Layer (sentence-transformers)
3. Retrieval System (FAISS)
4. Augmentation (context enrichment)
5. Generation (answer synthesis)

## Setup: Import All Components

Run this cell first to load all modules.

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers")

In [2]:
import sys, os
sys.path.append(os.path.abspath(".."))

In [4]:
# Import our 5 components
from knowledge_base import KnowledgeBase
from semantic_layer import SemanticLayer
from retrieval_system import RetrievalSystem
from augmentation import Augmentation
from generation import Generation
print("✅ All components imported successfully!")

✅ All components imported successfully!


## Component 1: Knowledge Base

Load PCAI dataset from pdf.

In [5]:

# Initialize KB with PDF and chunk settings
kb = KnowledgeBase(pdf_path="../data/hpe-pcai.pdf", chunk_size=200, chunk_overlap=50)

# Load PDF pages
documents = kb.load_pdf_data()

# Create sentence-preserving chunks
chunks = kb.create_chunks()

# Inspect stats
stats = kb.get_stats()
stats = kb.get_stats()
print(f"\n📊 Knowledge Base:")
print(f"   Pages: {stats['total_pages']}")
print(f"   Chunks: {stats['total_chunks']}")
print(f"   Source: {stats['source']}")


📚 Knowledge Base initialized

📥 Loading PDF: ../data/hpe-pcai.pdf
✅ Loaded 24 pages from PDF
✅ Created 57 text chunks

📊 Knowledge Base:
   Pages: 24
   Chunks: 57
   Source: ../data/hpe-pcai.pdf


## Component 2: Semantic Layer

Convert text to numerical embeddings for semantic comparison.

In [6]:
import random
import numpy as np

# =========================
# Random Preview: Chunks & Embeddings
# =========================

# Number of random samples to preview
N = 5

# Ensure embeddings exist
try:
    document_embeddings
except NameError:
    # Encode chunks if not done yet
    semantic = SemanticLayer()
    document_embeddings = semantic.encode_documents(kb.chunks)

# Random indices
random_indices = random.sample(range(len(kb.chunks)), min(N, len(kb.chunks)))

print(f"\n📄 Random preview of {N} chunks with embeddings:\n")

for i, idx in enumerate(random_indices):
    chunk = kb.chunks[idx]
    emb = document_embeddings[idx]
    
    print(f"--- Sample {i+1} (Index {idx}) ---")
    print(f"Page: {chunk['page']}")
    print(f"Text (first 200 chars): {chunk['chunk'][:200]}...")
    print(f"Embedding shape: {emb.shape}")
    print(f"Embedding sample (first 10 values): {emb[:10]}")
    print("-" * 70)


COMPONENT 2: Semantic Layer initializing...
   Loading model: all-MiniLM-L6-v2
   (First run downloads 22MB - takes 30 seconds)
✅ Model loaded! Embeddings dimension: 384

🔄 Encoding 57 document chunks to embeddings...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Embeddings created: (57, 384)

📄 Random preview of 5 chunks with embeddings:

--- Sample 1 (Index 23) ---
Page: 9
Text (first 200 chars): Security Framework Components Security Framework Components Keycloak Keycloak Integration: Enterprise-grade identity and access management system implementing OIDC provider capabilities. Oauth2 Proxy:...
Embedding shape: (384,)
Embedding sample (first 10 values): [-0.08842835  0.0362228  -0.0836657  -0.04593932  0.01984975 -0.05168671
  0.10059242  0.0336007   0.00963222 -0.01973345]
----------------------------------------------------------------------
--- Sample 2 (Index 39) ---
Page: 16
Text (first 200 chars): HPE sends an email confirmation to the customer of successful registration within the My HPE Software Center. 6 . The customer provides HPE the license key files. 7 . The installer drags/drops the fil...
Embedding shape: (384,)
Embedding sample (first 10 values): [-0.07583776  0.02418172  0.01349287 -0.11196134  0.01885809  0.06329645
  0.

## Component 3: Retrieval System

Build FAISS index for fast similarity search.

In [7]:
# Initialize Retrieval System
retrieval = RetrievalSystem(dimension=384)

# Build searchable index
retrieval.build_index(document_embeddings, kb.chunks)

print("\n✅ Retrieval system ready to search!")

🔍 COMPONENT 3: Retrieval System initialized (dim=384)

🔨 Building FAISS index...
   Documents: 57
   Embedding shape: (57, 384)
✅ Index built with 57 vectors

✅ Retrieval system ready to search!


In [8]:
# Inspect retrieval output
results = retrieval.search(semantic.encode_query("What are pcai core components"), top_k=3)
print("Results structure:\n")
for i, r in enumerate(results):
    print(f"Result {i}:")
    print(r)
    print("-" * 50)



🔎 Searching for top 3 relevant documents...
✅ Retrieved 3 documents
Results structure:

Result 0:
{'rank': 1, 'document': {'page': 1, 'chunk': 'HPE Private Cloud AI 1.5 Administration Guide HPE Private Cloud AI 1.5 Administration Guide Part Number: 20-PCAI-UG-ED1 Published: September 2025 Edition: 1.5'}, 'similarity': 0.4387816354426174, 'distance': 1.2790379524230957}
--------------------------------------------------
Result 1:
{'rank': 2, 'document': {'page': 3, 'chunk': 'Table of contents Table of contents Overview of the HPE Private Cloud AI engineered system Private Cloud AI Stack Developer System HPE AI Essentials: Core Concepts Solution components Optimum environment Airflow requirements Power requirements Space requirements Temperature requirements Firewall and port requirements Networking requirements Licensing requirements Activating the NVIDIA software subscription Solution configurations Configuration sizes Private Cloud AI User roles Signing in to the HPE GreenLake platfo

## Test the Complete RAG Pipeline

Now let's ask a question and see all 5 components work together!

In [8]:
def ask_question(question, top_k=3):
    """Complete RAG pipeline for a question using PCAI PDF knowledge base"""
    
    print("=" * 70)
    print(f"❓ QUESTION: {question}")
    print("=" * 70)
    
    # COMPONENT 2: Encode query
    print("\n[Step 1] Encoding question...")
    query_embedding = semantic.encode_query(question)
    
    # COMPONENT 3: Retrieve relevant chunks
    print("[Step 2] Searching knowledge base...")
    results = retrieval.search(query_embedding, top_k=top_k)
    
    print(f"\n📚 Retrieved Top {top_k} Chunks:")
    for r in results:
        doc_page = r['document']['page']
        doc_text = r['document']['chunk']
        print(f"  • Page {doc_page} (similarity: {r['similarity']:.1%})")
        print(f"    Text snippet: {doc_text[:200]}...\n")
    
    # COMPONENT 4: Augment with context
    print("[Step 3] Creating enriched context...")
    augmentor = Augmentation()
    enriched = augmentor.create_context(question, results)
    
    # COMPONENT 5: Generate answer
    print("[Step 4] Generating answer...")
    generator = Generation(model_name='llama3.2:3b', use_llm=True)
    answer = generator.generate(enriched, results)
    
    # Display answer
    print("\n" + "=" * 70)
    print("💬 ANSWER:")
    print("=" * 70)
    print(answer)
    print("=" * 70)
    
    return answer

# Example usage
ask_question("What are pcai core tools")



❓ QUESTION: What are pcai core tools

[Step 1] Encoding question...
[Step 2] Searching knowledge base...

🔎 Searching for top 3 relevant documents...
✅ Retrieved 3 documents

📚 Retrieved Top 3 Chunks:
  • Page 7 (similarity: 47.9%)
    Text snippet: HPE HPE AI Essentials AI Essentials : Core Concepts : Core Concepts HPE AI Essentials Software is part of the HPE Private Cloud AI software and data foundation layer that accelerates the time from AI ...

  • Page 8 (similarity: 47.7%)
    Text snippet: AI assistants for the fastest path to inference with unparalleled performance. HPE AI Essentials Software currently includes the following pre-packaged NVIDIA NIM: NVIDIA NIM for GPU accelerated NVIDI...

  • Page 8 (similarity: 47.3%)
    Text snippet: NVIDIA Blueprints NVIDIA Blueprints are predefined, customizable AI workflows from NVIDIA that can assist you in creating and deploying generative AI applications. Blueprints are designed for you to d...

[Step 3] Creating enriched context...

'Based on the provided context from Page 7 of the PDF, PCAI core tools are not explicitly mentioned as a specific component of HPE AI Essentials Software. However, it can be inferred that some of the pre-packaged tools and frameworks included in HPE AI Essentials Software, such as NVIDIA NIM (NVIDIA Institute for Machine Learning) models, are part of the core components.\n\nSpecifically, the following tools are mentioned:\n\n1. NVIDIA NIM: A set of pre-packaged models for inference with unparalleled performance.\n2. Mistral 7B and Llama 3 models: OpenAI-compatible APIs for inference.\n\nThese tools can be considered as part of the core components of HPE AI Essentials Software, which provides a ready-to-run set of AI and open-source tools to build end-to-end GenAI solutions.\n\n======================================================================\n📚 SOURCES:\n======================================================================\n\n[1] Page 7 (Similarity: 47.9%)\n    Preview: HPE HPE A

## Try More Questions!

Test the system with different queries.

In [9]:
# Try these questions or write your own!

ask_question("What is pcai engineered system for?")

❓ QUESTION: What is pcai engineered system for?

[Step 1] Encoding question...
[Step 2] Searching knowledge base...

🔎 Searching for top 3 relevant documents...
✅ Retrieved 3 documents

📚 Retrieved Top 3 Chunks:
  • Page 8 (similarity: 46.9%)
    Text snippet: NVIDIA Blueprints NVIDIA Blueprints are predefined, customizable AI workflows from NVIDIA that can assist you in creating and deploying generative AI applications. Blueprints are designed for you to d...

  • Page 1 (similarity: 45.6%)
    Text snippet: HPE Private Cloud AI 1.5 Administration Guide HPE Private Cloud AI 1.5 Administration Guide Part Number: 20-PCAI-UG-ED1 Published: September 2025 Edition: 1.5...

  • Page 3 (similarity: 45.4%)
    Text snippet: Table of contents Table of contents Overview of the HPE Private Cloud AI engineered system Private Cloud AI Stack Developer System HPE AI Essentials: Core Concepts Solution components Optimum environm...

[Step 3] Creating enriched context...
⚡ COMPONENT 4: Augmentation in

'Based on the provided context, PCAI (Private Cloud AI) is an engineered system for:\n\n1. Managing and deploying NVIDIA Blueprints\n2. Providing a comprehensive infrastructure for building and running AI applications\n3. Offering isolated, secure notebook environments for each user with dedicated computational resources.\n\nOverall, PCAI appears to be a solution for businesses to transform their unique expertise and data into groundbreaking AI solutions that drive competitive advantage and unlock new opportunities in their respective fields.\n\n======================================================================\n📚 SOURCES:\n======================================================================\n\n[1] Page 8 (Similarity: 46.9%)\n    Preview: NVIDIA Blueprints NVIDIA Blueprints are predefined, customizable AI workflows from NVIDIA that can assist you in creating and deploying generative AI ...\n\n[2] Page 1 (Similarity: 45.6%)\n    Preview: HPE Private Cloud AI 1.5 Administration Gui

In [10]:
ask_question("does HPE offer supportf or AI products on delivery")

❓ QUESTION: does HPE offer supportf or AI products on delivery

[Step 1] Encoding question...
[Step 2] Searching knowledge base...

🔎 Searching for top 3 relevant documents...
✅ Retrieved 3 documents

📚 Retrieved Top 3 Chunks:
  • Page 18 (similarity: 54.8%)
    Text snippet: 14 . Access the licensing portal through the email link Solution configurations Solution configurations WARNING WARNING The listed versions are the minimum required; do not downgrade, modify, or repla...

  • Page 7 (similarity: 54.6%)
    Text snippet: HPE HPE AI Essentials AI Essentials : Core Concepts : Core Concepts HPE AI Essentials Software is part of the HPE Private Cloud AI software and data foundation layer that accelerates the time from AI ...

  • Page 2 (similarity: 53.4%)
    Text snippet: HPE HPE Private Cloud AI Private Cloud AI 1.5 Administration Guide 1.5 Administration Guide Abstract Abstract This guide is the primary reference for Cloud Administrators, AI Administrators, and AI Us...

[Step 3] C

'Based on the provided context, it appears that HPE offers support for AI products through various means.\n\nOn Page 18 of the PDF, it is mentioned that the HPE Private Cloud AI solution is available in different configurations, including a configuration labeled as "HPE Private Cloud AI". This suggests that HPE provides some level of support or documentation for this specific configuration.\n\nAdditionally, on Page 7, the document mentions HPE AI Essentials Software, which includes open-source tools and frameworks specifically designed for AI workloads. The document also provides information on how to use and access this software, indicating that HPE may offer support or resources for users who need help with this product.\n\nOn Page 2, the document states that it is a primary reference guide for Cloud Administrators, AI Administrators, and AI Users who work with the HPE Private Cloud AI AI solution. This suggests that HPE provides documentation and support materials for the HPE Privat

In [11]:
# Your own question here!
ask_question("Who is customer support engineer, doe they helo with AI platform")

❓ QUESTION: Who is customer support engineer, doe they helo with AI platform

[Step 1] Encoding question...
[Step 2] Searching knowledge base...

🔎 Searching for top 3 relevant documents...
✅ Retrieved 3 documents

📚 Retrieved Top 3 Chunks:
  • Page 23 (similarity: 46.5%)
    Text snippet: performance and workload management tasks such as: Managing AI Essentials Software resources Managing AI model deployment and versioning Monitoring AI performance and fine-tuning models and GPU resour...

  • Page 2 (similarity: 46.1%)
    Text snippet: HPE HPE Private Cloud AI Private Cloud AI 1.5 Administration Guide 1.5 Administration Guide Abstract Abstract This guide is the primary reference for Cloud Administrators, AI Administrators, and AI Us...

  • Page 3 (similarity: 45.7%)
    Text snippet: Table of contents Table of contents Overview of the HPE Private Cloud AI engineered system Private Cloud AI Stack Developer System HPE AI Essentials: Core Concepts Solution components Optimum environm.

'Based on the provided context, a Customer Support Engineer may not be directly involved with helping customers with an AI platform.\n\nThe relevant section in the PDF is related to the "Private Cloud AI User" role, which includes tasks such as developing and deploying AI applications, managing model fine-tuning, and ensuring compliance with ethical use. This suggests that users with this role are typically researchers and data scientists who work closely with the AI platform.\n\nHowever, the Customer Support Engineer\'s role is not explicitly mentioned in the provided context. It may be related to general support or troubleshooting for HPE products and services, but there is no specific mention of AI platforms or solutions.\n\nTherefore, it appears that customer support engineers may not typically interact directly with customers who are using an AI platform, as this role seems to be more focused on developing and deploying AI applications within the Private Cloud AI environment.\n\n=

## Interactive Mode

Keep asking questions!

In [ ]:
# Interactive questioning
while True:
    user_question = input("\nYour question (or 'quit' to stop): ").strip()
    
    if user_question.lower() in ['quit', 'exit', 'q']:
        print("👋 PDF RAG here to answer")
        break
    
    if user_question:
        ask_question(user_question)
    else:
        print("Please enter a question!")


Your question (or 'quit' to stop):  what are solution components,  how are they used


❓ QUESTION: what are solution components,  how are they used

[Step 1] Encoding question...
[Step 2] Searching knowledge base...

🔎 Searching for top 3 relevant documents...
✅ Retrieved 3 documents

📚 Retrieved Top 3 Chunks:
  • Page 9 (similarity: 43.5%)
    Text snippet: and permission management Features sophisticated data source connection management with security controls Allows administrators to configure and customize included applications through the user interf...

  • Page 10 (similarity: 42.5%)
    Text snippet: Entity Entity Component Details Component Details Small, Medium and Large Small, Medium and Large Developer System Developer System Software Built on the HPE GreenLake Cloud Platform GreenLake Cloud P...

  • Page 8 (similarity: 40.6%)
    Text snippet: NVIDIA Blueprints NVIDIA Blueprints are predefined, customizable AI workflows from NVIDIA that can assist you in creating and deploying generative AI applications. Blueprints are designed for you to d...

[Step 3] Cre